# Progessing through Medallion Architecture


This notebook connects updates the information in the different tables of our medallion architecture in **watsonx.data**. We start buy adding any new records from our original Car_data table into our bronze layer table.

We then compare the new records in our bronze table to the existing records in our silver table.

If there are new, unsold cars records, we will add them to our silver table.

Lastly, we aggregate the origin country data and counts from each country. We then insert these summaries into our gold table.

## Connecting to our watonsx.data Presto Engine

In [ ]:
pip install --upgrade presto

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install presto-python-client

import os
import IPython

if "RESTARTED" not in os.environ:
    os.environ["RESTARTED"] = "1"
    IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart_and_run_all()"))

In [ ]:
#Neccessary Imports
import os
import prestodb


#Authentication for Data Lakehouse
username="ibmlhapikey"
password="amFuc3NlbEBzbGFjLnN0YW5mb3JkLmVkdTpiMXRzZ3RhSlFSbFlDTXUzUEFuQUxjNVhsYTN3Uzl6VmpTNG1FcHJ4"
hostname="ibm-lh-lakehouse-presto-01-presto-svc-ibm-cpd-instance.apps.lakehouse.f1dp.p1.openshiftapps.com"
portnumber="443"

with prestodb.dbapi.connect(
host=hostname ,
port=portnumber,
user=username,
catalog='iceberg_data',
schema= 'test_schema' ,
http_scheme='https',
auth=prestodb.auth.BasicAuthentication(username, password)
) as conn :
#Specify starting table
    cur = conn.cursor()
    cur.execute('select * from iceberg_data.test_schema.car_data limit 10') #Raw data coming from Car_Data Table
    rows = cur.fetchall()
    print(rows) #Confirming expected out



## Updating our bronze layer table
Addtionally, we preform a few count queries to compare previously existing unsold records (silver) to new unsold records (bronze unique).

In [ ]:
    #Addition to bronze table from original Car_data table
    cur.execute("INSERT INTO iceberg_data.medallion.bronze SELECT * FROM iceberg_data.test_schema.car_data AS a WHERE NOT EXISTS (SELECT * FROM iceberg_data.medallion.bronze AS b WHERE a.id = b.id)")

In [ ]:
    #Counting total records in bronze table
    cur.execute("SELECT COUNT (*) FROM iceberg_data.medallion.bronze")
    brozne_count = cur.fetchall()
    print("Bronze Total Count:", (brozne_count))


    #Counting only unique records in bronze table
    cur.execute("SELECT COUNT (*) FROM iceberg_data.medallion.bronze WHERE carrier = ''")
    bronze_unique = cur.fetchall()
    print("Bronze Unique Count:", (bronze_unique))


    #Counting total records in silver table
    cur.execute("SELECT COUNT (*) FROM iceberg_data.medallion.silver")
    silver_count = cur.fetchall()
    print("Silver Total Count:", (silver_count))

## Adding new unsold records to silver table
If there is a difference between the unique count in the bronze table and the total count in the silver table, we will execute the next cell to add those new records to our silver table.
Additionally, we perform a fresh count of the silver table records to ensure it matches the unique count in our bronze table.

In [ ]:
    #Inserting new unsold cars records into our silver table.
    cur.execute("INSERT INTO iceberg_data.medallion.silver SELECT * FROM iceberg_data.medallion.bronze AS a WHERE carrier = '' AND NOT EXISTS (SELECT * FROM iceberg_data.medallion.silver AS b WHERE a.id = b.id)")

In [ ]:
    cur.execute("SELECT COUNT (*) FROM iceberg_data.medallion.silver")
    silver_new_count = cur.fetchall()
    print("Silver UpdatedTotal Count:", (silver_new_count))

## Aggregating silver table data
Here, we will get a summary for the amount of records that exist for the different values in the 'origin' column of our silver table.

In [ ]:
    #List the different origin countries and the number of records that exist for each
    cur.execute("SELECT origin, COUNT(*) FROM iceberg_data.medallion.silver GROUP BY origin")
    silver_origin_count = cur.fetchall()
    print(silver_origin_count)

## Populating gold table
Finally, based our aggregation of our silver table, we will insert these summaries into our gold table.

In [ ]:
    #For demo purposes, to aviod duplicates we will delete the data in our gold table and insert fresh summaries from our silver table
    cur.execute("DELETE FROM iceberg_data.medallion.gold")
    cur.execute("INSERT INTO iceberg_data.medallion.gold SELECT origin, COUNT(*) FROM iceberg_data.medallion.silver GROUP BY origin")